In [1]:
import numpy as np
import pickle
import sys

from multiprocessing import Pool
from collections import Counter
from tqdm import tqdm

In [2]:
# Calculate subsampling rates
with open(r'D:\dstore\nlp\w2v\wcounts', 'r', encoding='utf-8') as f:
    wcounts = [x.split() for x in f.readlines()]
wcounts = {x[1]: int(x[0]) for x in wcounts if int(x[0]) > 0}
excess = {x[0]: (10000 / x[1]) ** 0.5 for x in wcounts.items() if x[1] > 10000}

del wcounts

In [3]:
#Subsampling Excessively Common Words

rng = np.random.default_rng()
for ft in range(1, 5):

    with open(f'D:\\dstore\\nlp\\w2v\\nwiki5_{ft}', 'r', encoding='utf-8') as f:
        words1 = [x.strip('\n') for x in f.readlines()]

    cwords = []
    while words1:
        hold = words1.pop().split()
        slen = len(hold) - 1
        for i, x in enumerate(hold[::-1]):
            if x in excess:
                if rng.random() > excess[x]:
                    hold.pop(slen - i)
        if len(hold) > 3:
            cwords.append(hold)

    with open(f'D:\\dstore\\nlp\\w2v\\vtrain{ft}', 'wb') as f:
        pickle.dump(cwords, f)

del cwords, words1

In [2]:
#Tallying words in subsampled data
counts = Counter()

for x in range(1, 5):
    with open(f'D:\\dstore\\nlp\\w2v\\vtrain{x}', 'rb') as f:
        word_array = pickle.load(f)
    for y in word_array:
        for z in y:
            counts[z] += 1

del word_array
all_words = np.array([x for x in counts])

with open('word_list', 'wb') as f:
    pickle.dump(all_words, f)

In [3]:
#Generating word index dictionary
with open(f'word_list', 'rb') as f:
    all_words = pickle.load(f)

np.random.shuffle(all_words)
wordids = {x: i  for i, x in enumerate(all_words)}

with open(f'word_list_i', 'wb') as f:
    pickle.dump(wordids, f)